In [ ]:
import sys
!{sys.executable} -m pip install sympy
!{sys.executable} -m pip install plotly

# Task 1 (40 points):
Based on Planck’s law for the radiance of the Sun, assuming a temperature $T$ of $5775\text{K}$, create four plots, which show
A. The spectral irradiance versus photon wavelength
B. The spectral irradiance versus photon energy
C. The photon flux versus photon wavelength
D. The photon flux versus energy.

You can convert the radiance of the object to the irradiance by multiplying the radiance by the steradian angle of the Sun’s disc as it is observed on Earth. That value is $6.794\cdot10^{-5}\text{sr}$. At the end of Task 1, you should have four plots that look similar to the ones on the PV Lighthouse web site:
https://www2.pvlighthouse.com.au/resources/courses/altermatt/The%20Solar%20Spectrum/Intensity%20or%20photon%20flux.aspx

The axis labels should match the ones on PV Lighthouse, except that you should not convert the photon flux to
$\frac{\text{mA}}{\text{cm}^2\cdot\text{eV}}$, but leave it as $\frac{\text{Number of photons}}{\text{s}\cdot\text{m}^2\cdot\text{eV}}$.

In [31]:
from numpy import sqrt, exp, linspace
from sympy import symbols
from QMFunctions import *
from plotly.express import scatter
from pandas import DataFrame
import scipy.constants as sc

sun_steradian_angle = 6.794*10**(-5) # in sr
h = sc.h
k_b = sc.k
#h and k_b multiplied by q = 1.6022*10**(-19) to convert the eV's in their units to Joules, so the irradiance has units of Watts per m^3

def planck_law(T, Lambda = None, f = None):
  if Lambda is not None:
    irradiance = (2*h*(c**2)/((exp(h*c/(Lambda*k_b*T))-1)*Lambda**5))*sun_steradian_angle
    return irradiance*10**(-9) # convert units from W/m^3 to W/(m^2*nm) to match units used on pvlighthouse website

  if f is not None:
    return (2*h*f**3)/((exp(h*f/(k_b*T))-1)*c**2)*sun_steradian_angle


A. The spectral irradiance versus photon wavelength

In [32]:
wavelength = list(range(280,3000))
irradiance = []

for i in wavelength:
  irradiance.append(planck_law(T = 5775, Lambda = i*10**(-9)))

scatter(x = wavelength, y = irradiance)

In [26]:
df = DataFrame(wavelength, irradiance)

In [27]:
df

,0
5.270234e-34,280
5.195612e-34,281
5.122307e-34,282
5.050289e-34,283
4.979533e-34,284
...,...
4.026009e-38,2995
4.020636e-38,2996
4.015272e-38,2997
4.009918e-38,2998
